<a href="https://colab.research.google.com/github/fatemafaria142/Comparative-Analysis-of-Diverse-Large-Language-Models-in-Story-Generation/blob/main/Story_Generation_using_TinyLlama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Install Required Packages**

In [1]:
!pip install accelerate peft bitsandbytes transformers trl datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.9/78.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.2 MB/s eta 0:00:00


# **Load the required packages**

In [2]:
import torch
from datasets import load_dataset, Dataset
from peft import LoraConfig, AutoPeftModelForCausalLM
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from trl import SFTTrainer
import os

### **Dataset Link:** https://huggingface.co/datasets/nlpie/Llama2-MedTuned-Instructions?row=0

In [3]:
dataset="AtlasUnified/atlas-storyteller"
model_id="TinyLlama/TinyLlama-1.1B-Chat-v1.0"
output_model="tinyllama-Story-v1"

# **Dataset preparation**

In [4]:
def prepare_train_data(data_id, num_samples=1000):
    data = load_dataset(data_id, split="train")

    # Take the first 1000 rows
    data = data.select(range(min(num_samples, len(data))))
    instructions_template = "Imagine you are the author of this story. Your task is to continue the narrative and unfold the plot. Introduce new characters, unexpected twists, and exciting events.Feel free to unleash your creativity and have fun crafting the next part of the story!"
    data_df = data.to_pandas()
    data_df["text"] = data_df[["Story",]].apply(lambda x: "user\n" + instructions_template + "  \n Story\n" + x["Story"] + "\nassistant\n"  "\n", axis=1)
    data = Dataset.from_pandas(data_df)

    return data



In [5]:
data = prepare_train_data(dataset, num_samples=1000)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [6]:
data

Dataset({
    features: ['id', 'Story', 'text'],
    num_rows: 1000
})

In [7]:
data[0]

{'id': 'seed_task_0',
 'Story': 'In the bustling city of New York, where the neon lights flickered and the sound of traffic reverberated through the streets, lived a man named Ethan Sullivan. With his chiseled jawline, piercing blue eyes, and a physique carved by years of training, Ethan possessed an air of mystery that intrigued those who crossed his path. He lived a solitary life in a modest apartment, spending his days as an accountant, his nights cloaked in shadows and secrecy.  One fateful morning, as the city awoke to the rhythmic beat of its own heartbeat, Ethan received a cryptic message on a burner phone. It simply stated, "They\'re coming for you." His heart quickened, and a wave of apprehension washed over him. Who were "they," and why were they after him? Ethan\'s blood ran cold, fueling a surge of adrenaline that urged him to take action.  Without a moment\'s hesitation, Ethan gathered his meager belongings and made his way to a hidden room beneath his apartment. The room,

In [8]:
data[2]

{'id': 'seed_task_2',
 'Story': 'In the midst of a bustling metropolis teeming with skyscrapers that seemingly touched the heavens, nestled a small and unassuming garage. It belonged to Ethan, a young and brilliant engineer consumed by his insatiable thirst for innovation and adventure. With grease-stained hands and a mind full of ideas, he spent every waking hour tinkering and crafting marvelous inventions that could turn dreams into reality.  One fateful afternoon, while rummaging through a box filled with ancient blueprints, Ethan stumbled upon a peculiar schematic. It depicted a colossal machine, a menacing giant robot with gleaming steel limbs and an aura of unstoppable power. Intrigued and fascinated, he couldn\'t resist the temptation to bring this mechanical behemoth to life.  Days turned into weeks, and nights melted away as Ethan toiled tirelessly in his secluded workshop, pouring every ounce of his skill and determination into the construction of his masterpiece. Finally, af

## **We have to model the Model (not the base version)**

In [9]:
def get_model_and_tokenizer(mode_id):

    tokenizer = AutoTokenizer.from_pretrained(mode_id)
    tokenizer.pad_token = tokenizer.eos_token
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype="float16", bnb_4bit_use_double_quant=True
    )
    model = AutoModelForCausalLM.from_pretrained(
        mode_id, quantization_config=bnb_config, device_map="auto"
    )
    model.config.use_cache=False
    model.config.pretraining_tp=1
    return model, tokenizer

In [10]:
model, tokenizer = get_model_and_tokenizer(model_id)

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

# **Setting up the LoRA**

In [11]:
peft_config = LoraConfig(
        r=8, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM"
    )

In [15]:
training_arguments = TrainingArguments(
        output_dir=output_model,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        optim="paged_adamw_32bit",
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=10,
        num_train_epochs=1,
        max_steps=250,
        fp16=True,
        # push_to_hub=True
    )

In [16]:
trainer = SFTTrainer(
        model=model,
        train_dataset=data,
        peft_config=peft_config,
        dataset_text_field="text",
        args=training_arguments,
        tokenizer=tokenizer,
        packing=False,
        max_seq_length=1024
    )

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [17]:
trainer.train()

Step,Training Loss
10,1.651800
20,1.619900
30,1.562600
40,1.515200
50,1.460100
60,1.512100
70,1.464100
80,1.471200
90,1.458100
100,1.454600


TrainOutput(global_step=250, training_loss=1.4707638320922851, metrics={'train_runtime': 746.8045, 'train_samples_per_second': 1.339, 'train_steps_per_second': 0.335, 'total_flos': 6251227549212672.0, 'train_loss': 1.4707638320922851, 'epoch': 1.0})

# **Merging the LoRA with the base model**

In [18]:
from peft import AutoPeftModelForCausalLM, PeftModel
from transformers import AutoModelForCausalLM
import torch
import os

model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, load_in_8bit=False,
                                             device_map="auto",
                                             trust_remote_code=True)

model_path = "/content/tinyllama-Story-v1/checkpoint-250"

peft_model = PeftModel.from_pretrained(model, model_path, from_transformers=True, device_map="auto")

model = peft_model.merge_and_unload()

In [19]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head)

# **Inference from the LLM for some examples**

In [23]:
def formatted_prompt(instruction, input)-> str:
    return f"<|im_start|>user\n{instruction}<|im_end|>\n<|im_start|>Story\n{input}<|im_end|>\n<|im_start|>assistant:"

In [27]:
from transformers import GenerationConfig
from time import perf_counter

def generate_response(instruction, user_input):

    prompt = formatted_prompt(instruction, user_input)

    inputs = tokenizer([prompt], return_tensors="pt")
    generation_config = GenerationConfig(
        penalty_alpha=0.6,
        do_sample=True,
        top_k=5,
        temperature=0.5,
        repetition_penalty=1.2,
        max_new_tokens=1024,
        pad_token_id=tokenizer.eos_token_id
    )
    start_time = perf_counter()

    inputs = tokenizer(prompt, return_tensors="pt").to('cuda')

    outputs = model.generate(**inputs, generation_config=generation_config)
    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()  # Remove leading and trailing whitespaces
    print(decoded_output)
    output_time = perf_counter() - start_time
    print(f"Time taken for inference: {round(output_time, 2)} seconds")

#### **Example No: 1**

In [28]:
instructions_template = "Imagine you are the author of this story. Your task is to continue the narrative and unfold the plot. Introduce new characters, unexpected twists, and exciting events.Feel free to unleash your creativity and have fun crafting the next part of the story!"
user_input = 'Once upon a time in the vibrant city of Dhaka, there were three inseparable friends - Hridoy, Mukaffi, and Wasi. Their friendship was a testament to the bond that can be formed in the most unexpected of places.'
generate_response(instruction=instructions_template, user_input=user_input)

<|im_start|>user
Imagine you are the author of this story. Your task is to continue the narrative and unfold the plot. Introduce new characters, unexpected twists, and exciting events.Feel free to unleash your creativity and have fun crafting the next part of the story!<|im_end|>
<|im_start|>Story
Once upon a time in the vibrant city of Dhaka, there were three inseparable friends - Hridoy, Mukaffi, and Wasi. Their friendship was a testament to the bond that can be formed in the most unexpected of places.<|im_end|>
<|im_start|>assistant: 
Welcome back to our continuation of "The Three Friends." In this second installment, we follow the journey of these close-knit friends as they navigate through life's ups and downs together. The streets of Dhaka come alive with their adventures and discoveries, while also holding secrets that threaten to tear them apart once again. Let the pages take you on an enchanting journey filled with laughter, heartbreak, and triumph. 
Chapter One 
As the sun se

#### **Example No: 2**

In [29]:
instructions_template = "Imagine you are the author of this story. Your task is to continue the narrative and unfold the plot. Introduce new characters, unexpected twists, and exciting events.Feel free to unleash your creativity and have fun crafting the next part of the story!"
user_input = '''In Dhaka's vibrant Ahsanullah University, Asif Fahim, Mukaffi, and Morshed forged a lasting bond. As students, they navigated the complexities of science and technology, each with unique dreams.'''
generate_response(instruction=instructions_template, user_input=user_input)

<|im_start|>user
Imagine you are the author of this story. Your task is to continue the narrative and unfold the plot. Introduce new characters, unexpected twists, and exciting events.Feel free to unleash your creativity and have fun crafting the next part of the story!<|im_end|>
<|im_start|>Story
In Dhaka's vibrant Ahsanullah University, Asif Fahim, Mukaffi, and Morshed forged a lasting bond. As students, they navigated the complexities of science and technology, each with unique dreams.<|im_end|>
<|im_start|>assistant: Welcome back to our continuation of "The Journey of Science" series! In this installment, we delve deeper into the world of scientific discovery and its impact on society. Let us embark upon another journey where our protagonist Asif Fahim, Mukaffi, and Morshed strive to make an imprint in the annals of history through their groundbreaking research. 

Asif was a brilliant student with a passion for physics, determined to unravel the mysteries of the universe. His curio

#### **Example No: 3**

In [30]:
instructions_template = "Imagine you are the author of this story. Your task is to continue the narrative and unfold the plot. Introduce new characters, unexpected twists, and exciting events.Feel free to unleash your creativity and have fun crafting the next part of the story!"
user_input = '''In a happy town where everyone laughed and had big dreams, there were two siblings named Fasabbih and Mariya. Fasabbih, the little brother, always worked quietly towards his goals, while Mariya, his big sister, had a bright and energetic personality that made their small town feel alive.'''
generate_response(instruction=instructions_template, user_input=user_input)

<|im_start|>user
Imagine you are the author of this story. Your task is to continue the narrative and unfold the plot. Introduce new characters, unexpected twists, and exciting events.Feel free to unleash your creativity and have fun crafting the next part of the story!<|im_end|>
<|im_start|>Story
In a happy town where everyone laughed and had big dreams, there were two siblings named Fasabbih and Mariya. Fasabbih, the little brother, always worked quietly towards his goals, while Mariya, his big sister, had a bright and energetic personality that made their small town feel alive.<|im_end|>
<|im_start|>assistant: Welcome back to the continuation of our story about Fasabbih and Mariya. The sun blazed down on their quiet village as they sat together at lunchtime, sipping their juice boxes in anticipation of another day of schoolwork. Fasabbih was an avid reader who loved nothing more than curling up with a good book under a cozy blanket. His eyes gleamed with excitement as he flipped thr